# fer_generator.py的代码测试与调试
流程上，主要就是原fer2013拆分成三个集合，把集合转为tfRecord，从tfrecord读出，供训练使用。

## 常见问题：
图片像素对应的数据类型的选择：
我的int默认是8字节，因为PIL不太支持，所以改成了4字节np.int32。其实因为是灰度图，8位255足矣


## def split_data():

In [1]:
import tensorflow as tf
from PIL import Image
import os
import random
import pandas as pd
import numpy as np
import fer_config as config

In [3]:
data = pd.read_csv(config.data_path)

<class 'pandas.core.frame.DataFrame'>


In [14]:
#emotion是label， pixels是图像， Usage是数据集
images = data['pixels']
labels = data['emotion']
usages = data['Usage']

img_train = []
img_valid = []
img_test = []
label_train = []
label_valid = []
label_test = []
#分三个集合
for i in range(len(images)):
    if usages[i] == 'Training':
        img_train.append(images[i])#一个一个的append，不然也没什么好方法吧。。。。
        label_train.append(labels[i])
    elif usages[i] == 'PrivateTest':
        img_valid.append(images[i])
        label_valid.append(labels[i])
    else:
        img_test.append(images[i])
        label_test.append(labels[i])
#打碎（成单个数据）之后再拼回来,data是dict形式，名称是列名，数据下标自动增长。
data_train = pd.DataFrame(data={'pixels':img_train,'emotion':label_train},columns=['pixels','emotion'])
print(data_train.shape, data_train['pixels'].dtype, data_train['emotion'].dtype)#(28709, 2),object,int64
data_valid = pd.DataFrame(data={'pixels':img_valid, 'emotion': label_valid}, columns=['pixels','emotion']) 
data_test = pd.DataFrame(data={'pixels':img_test, 'emotion': label_test}, columns=['pixels','emotion']) 
print(data_test['emotion'][:3])
print(type(data_test['pixels'][0]))#此时还是str
#仍然以csv格式保存
data_train.to_csv(config.train_data_path, encoding='utf_8_sig', index = False)#不额外加一个index列
#对比
# data_train.to_csv(config.train_data_path.split('.')[0]+'_with_index.csv', encoding='utf_8_sig', index = True)
data_valid.to_csv(config.valid_data_path, encoding='utf_8_sig', index = False)
data_test.to_csv(config.test_data_path, encoding = 'utf_8_sig', index = False)


4
image label usage's dtype: object int64 object
Training 0
(28709, 2) object int64
0    0
1    1
2    4
Name: emotion, dtype: int64
<class 'str'>


## 接口操作分解

In [12]:
print(type(data))
print(type(data),data.shape)
print(type(data['emotion']))
print(type(data['emotion'][0])==np.int64)
print(type(data['pixels'][0]),len(data['pixels'][0]))
print(data['emotion'][0])
print(data['emotion'][3])
print(usages[0],labels[0])
print('image label usage\'s dtype:',images.dtype,labels.dtype,usages.dtype)#字符串在这是object

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'> (35887, 3)
<class 'pandas.core.series.Series'>
True
<class 'str'> 8287
0


In [21]:
#dataframe的初始化，字典加list的形式
l1 = [1,2,3]
l2 = [2,3,4]
dataset = pd.DataFrame(data={'a':l1,'b':l2})
print(dataset)
# print(dataset[0])
print(dataset['a'])
print(dataset['b'])
print(dataset['b'][2])

   a  b
0  1  2
1  2  3
2  3  4
0    1
1    2
2    3
Name: a, dtype: int64
0    2
1    3
2    4
Name: b, dtype: int64
4


#  generate_images_and_labels和read_data
分割完数据集(train\valid\test)还需要处理，毕竟csv中还只是用字符串形式存的像素
先将三个csv的数据集分别都读出来，以备后用

In [28]:
def read_data(path,dbg_print = False):
    data = pd.read_csv(path)
    labels = data['emotion']
    #处理像素字符串。
    images = [np.array(p.split(" "),dtype= np.int32) for p in data['pixels']]#不支持64,也不需要，像素而已，可能我是64系统，int就是64了。
    
    if dbg_print:
        #测试打印
        print(data.shape)
        print(labels.shape)
        print('pixels shape:',data['pixels'].shape)#打印一下像素str的形状
        print('pixels str len:',len(data['pixels'][0]))#打印一下像素str的形状

        #处理为list后形状都一样，都是2304
        test_data = data['pixels'][0]
        test = np.array(test_data.split(" "),dtype= int)
        print(test.shape)
        print(test)
    
        test_data = data['pixels'][200]
        test = np.array(test_data.split(" "),dtype= int)
        print(test.shape)
        print(test)
    
    return images, labels


images, labels = read_data(config.train_data_path,dbg_print = True)
# print(len(images))#list
# print(len(images[0]))#单个数据，2304像素点
# print(labels.shape)


(28709, 2)
(28709,)
pixels shape: (28709,)
pixels str len: 8287
(2304,)
[ 70  80  82 ..., 106 109  82]
(2304,)
[134 136 121 ..., 169 171 171]


In [29]:
#generate images  labels 从文件中读取数据
images_train, labels_train = read_data(config.train_data_path)
images_valid, labels_valid = read_data(config.valid_data_path)
images_test, labels_test = read_data(config.test_data_path)

## 验证图片数据类型问题，被声明成32甚至64的“像素值”，读出来到底是多少位？如果不是8位，那么后边提取的时候8位的处理就错了。

注意事项（坑）：
bytes的len是2304，不是说真的是2304个byte的长，python对内存方面都是隐藏的，用惯了C++的我很不习惯。bytes是个对象，所以也无关内存大小。

核心操作.convert('L')，这个会把整型变成uint8，存文件之前就变了，后来读取img直接就tobytes了，所以按理说训练的时候直接decode拿uint8没问题。

关于打印：自己的例子，二进制很规整，/x01/x02  /xff，但是这个工程的图片，二进制很乱。
其实打印这确实也没法说，自己做几个简单的数字，bytes是很规整，一大了就看不懂了，但是实测数据没有任何问题

In [37]:
# print(labels_train.shape)
# print(labels_valid.shape)
# print(labels_test.shape)
# print(type(img_train))
# print(len(img_train))
# print(len(images_train))
# print(type(images_train))
# print(images_train[0])
# print(images_train[0].dtype)

#第一个实验：int32的数据转image，存文件，读文件，类型是什么？8位！
image = Image.fromarray(np.reshape(images_train[0], (config.img_width, config.img_height))).convert('L')
print(image)#能不能打印类型，找不到这个接口
image.save('img_save_test.png')
img = Image.open('img_save_test.png').convert('L')#灰度图
arr = np.array(img)
print('arr:',arr)
print('img:',img)#<PIL.Image.Image image mode=L size=48x48 at 0x7F2085391710>
# img.show()
img_raw = img.tobytes()
# print('img_raw:',img_raw)#byte打印，8位。
##毕竟不是C语言，这个len()得到的也是bytes对象给你的，所以没法做结论说内部存的就刚好是2304个字节长度，那只是元素数量。
#（不过限制了255,存储也的确是bytes）
print('type:',type(img_raw),'len(img_raw):',len(img_raw))
print('img_raw[0]:',img_raw[0])
print(type(img_raw[0]))


#bytes不是byte，
#他的数据成员打印出来仍然是“int”类型，bytes本身是一个类型，或者这个是一个对象。
#类型没得选#'bytes' object has no attribute 'type'
#bytes不允许传入255以上的数据，实际存储是8位
# a = bytes([1,2,3,4,255,256])
a = bytes([1,2,3,4,255])#bytes初始化，不允许传入255以上的数据
print('(type(a[0]):',type(a[0]))#这个类型int不代表bytes的存储类型
print(type(2.2))
print(a)#实际存储是8位，b'\x01\x02\x03\x04\xff'


<PIL.Image.Image image mode=L size=48x48 at 0x7EFDC6D7A438>
arr: [[ 70  80  82 ...,  52  43  41]
 [ 65  61  58 ...,  56  52  44]
 [ 50  43  54 ...,  49  56  47]
 ..., 
 [ 91  65  42 ...,  72  56  43]
 [ 77  82  79 ..., 105  70  46]
 [ 77  72  84 ..., 106 109  82]]
img: <PIL.Image.Image image mode=L size=48x48 at 0x7EFDC6DEFE80>
type: <class 'bytes'> len(img_raw): 2304
img_raw[0]: 70
type(img_raw): <class 'bytes'>
len(img_raw): 2304
<class 'int'>
(type(a[0]): <class 'int'>
<class 'float'>
b'\x01\x02\x03\x04\xff'


In [98]:
def generate(images, labels, images_path, labels_path):
    #这个地方需要路径的判断：
    images_path_is_exist = os.path.exists(images_path)
    if not images_path_is_exist:
        os.makedirs(images_path)
        print('create ',images_path)
    with open(labels_path, 'w', encoding='utf-8') as f:
        #转二维，转灰度图，按单个文件存储，文件名包含index和label。再把文件名存到txt。
        for i in range(len(images)):
#             if i == 0:#测试一个数据[ 70  80  82 ..., 106 109  82]
#                 print(np.array(images[i]))
            image = Image.fromarray(np.reshape(images[i], (config.img_width, config.img_height))).convert('L')
#             print('data type is ',np.array(img).dtype)#已经变8位了。
            img_path = str(i) + '_' + str(labels[i]) + '.png'
            image.save(images_path + img_path)
            f.write(img_path + ' ' + str(labels[i]))
            f.write('\n')
            print('the number of picture saved:',i)
    f.close()

generate(images_train, labels_train, config.image_train_path, config.label_train_path)
generate(images_valid, labels_valid, config.image_valid_path, config.label_valid_path)
generate(images_test, labels_test, config.image_test_path, config.label_test_path)


[ 70  80  82 ..., 106 109  82]
the number of picture saved: 0
the number of picture saved: 1
the number of picture saved: 2
the number of picture saved: 3
the number of picture saved: 4
the number of picture saved: 5
the number of picture saved: 6
the number of picture saved: 7
the number of picture saved: 8
the number of picture saved: 9
the number of picture saved: 10
the number of picture saved: 11
the number of picture saved: 12
the number of picture saved: 13
the number of picture saved: 14
the number of picture saved: 15
the number of picture saved: 16
the number of picture saved: 17
the number of picture saved: 18
the number of picture saved: 19
the number of picture saved: 20
the number of picture saved: 21
the number of picture saved: 22
the number of picture saved: 23
the number of picture saved: 24
the number of picture saved: 25
the number of picture saved: 26
the number of picture saved: 27
the number of picture saved: 28
the number of picture saved: 29
the number of pictu

the number of picture saved: 346
the number of picture saved: 347
the number of picture saved: 348
the number of picture saved: 349
the number of picture saved: 350
the number of picture saved: 351
the number of picture saved: 352
the number of picture saved: 353
the number of picture saved: 354
the number of picture saved: 355
the number of picture saved: 356
the number of picture saved: 357
the number of picture saved: 358
the number of picture saved: 359
the number of picture saved: 360
the number of picture saved: 361
the number of picture saved: 362
the number of picture saved: 363
the number of picture saved: 364
the number of picture saved: 365
the number of picture saved: 366
the number of picture saved: 367
the number of picture saved: 368
the number of picture saved: 369
the number of picture saved: 370
the number of picture saved: 371
the number of picture saved: 372
the number of picture saved: 373
the number of picture saved: 374
the number of picture saved: 375
the number

the number of picture saved: 721
the number of picture saved: 722
the number of picture saved: 723
the number of picture saved: 724
the number of picture saved: 725
the number of picture saved: 726
the number of picture saved: 727
the number of picture saved: 728
the number of picture saved: 729
the number of picture saved: 730
the number of picture saved: 731
the number of picture saved: 732
the number of picture saved: 733
the number of picture saved: 734
the number of picture saved: 735
the number of picture saved: 736
the number of picture saved: 737
the number of picture saved: 738
the number of picture saved: 739
the number of picture saved: 740
the number of picture saved: 741
the number of picture saved: 742
the number of picture saved: 743
the number of picture saved: 744
the number of picture saved: 745
the number of picture saved: 746
the number of picture saved: 747
the number of picture saved: 748
the number of picture saved: 749
the number of picture saved: 750
the number

the number of picture saved: 1132
the number of picture saved: 1133
the number of picture saved: 1134
the number of picture saved: 1135
the number of picture saved: 1136
the number of picture saved: 1137
the number of picture saved: 1138
the number of picture saved: 1139
the number of picture saved: 1140
the number of picture saved: 1141
the number of picture saved: 1142
the number of picture saved: 1143
the number of picture saved: 1144
the number of picture saved: 1145
the number of picture saved: 1146
the number of picture saved: 1147
the number of picture saved: 1148
the number of picture saved: 1149
the number of picture saved: 1150
the number of picture saved: 1151
the number of picture saved: 1152
the number of picture saved: 1153
the number of picture saved: 1154
the number of picture saved: 1155
the number of picture saved: 1156
the number of picture saved: 1157
the number of picture saved: 1158
the number of picture saved: 1159
the number of picture saved: 1160
the number of 

the number of picture saved: 1534
the number of picture saved: 1535
the number of picture saved: 1536
the number of picture saved: 1537
the number of picture saved: 1538
the number of picture saved: 1539
the number of picture saved: 1540
the number of picture saved: 1541
the number of picture saved: 1542
the number of picture saved: 1543
the number of picture saved: 1544
the number of picture saved: 1545
the number of picture saved: 1546
the number of picture saved: 1547
the number of picture saved: 1548
the number of picture saved: 1549
the number of picture saved: 1550
the number of picture saved: 1551
the number of picture saved: 1552
the number of picture saved: 1553
the number of picture saved: 1554
the number of picture saved: 1555
the number of picture saved: 1556
the number of picture saved: 1557
the number of picture saved: 1558
the number of picture saved: 1559
the number of picture saved: 1560
the number of picture saved: 1561
the number of picture saved: 1562
the number of 

...

the number of picture saved: 1780
the number of picture saved: 1781
the number of picture saved: 1782
the number of picture saved: 1783
the number of picture saved: 1784
the number of picture saved: 1785
the number of picture saved: 1786
the number of picture saved: 1787
the number of picture saved: 1788
the number of picture saved: 1789
the number of picture saved: 1790
the number of picture saved: 1791
the number of picture saved: 1792
the number of picture saved: 1793
the number of picture saved: 1794
the number of picture saved: 1795
the number of picture saved: 1796
the number of picture saved: 1797
the number of picture saved: 1798
the number of picture saved: 1799
the number of picture saved: 1800
the number of picture saved: 1801
the number of picture saved: 1802
the number of picture saved: 1803
the number of picture saved: 1804
the number of picture saved: 1805
the number of picture saved: 1806
the number of picture saved: 1807
the number of picture saved: 1808
the number of 

the number of picture saved: 2214
the number of picture saved: 2215
the number of picture saved: 2216
the number of picture saved: 2217
the number of picture saved: 2218
the number of picture saved: 2219
the number of picture saved: 2220
the number of picture saved: 2221
the number of picture saved: 2222
the number of picture saved: 2223
the number of picture saved: 2224
the number of picture saved: 2225
the number of picture saved: 2226
the number of picture saved: 2227
the number of picture saved: 2228
the number of picture saved: 2229
the number of picture saved: 2230
the number of picture saved: 2231
the number of picture saved: 2232
the number of picture saved: 2233
the number of picture saved: 2234
the number of picture saved: 2235
the number of picture saved: 2236
the number of picture saved: 2237
the number of picture saved: 2238
the number of picture saved: 2239
the number of picture saved: 2240
the number of picture saved: 2241
the number of picture saved: 2242
the number of 

the number of picture saved: 2657
the number of picture saved: 2658
the number of picture saved: 2659
the number of picture saved: 2660
the number of picture saved: 2661
the number of picture saved: 2662
the number of picture saved: 2663
the number of picture saved: 2664
the number of picture saved: 2665
the number of picture saved: 2666
the number of picture saved: 2667
the number of picture saved: 2668
the number of picture saved: 2669
the number of picture saved: 2670
the number of picture saved: 2671
the number of picture saved: 2672
the number of picture saved: 2673
the number of picture saved: 2674
the number of picture saved: 2675
the number of picture saved: 2676
the number of picture saved: 2677
the number of picture saved: 2678
the number of picture saved: 2679
the number of picture saved: 2680
the number of picture saved: 2681
the number of picture saved: 2682
the number of picture saved: 2683
the number of picture saved: 2684
the number of picture saved: 2685
the number of 

the number of picture saved: 3042
the number of picture saved: 3043
the number of picture saved: 3044
the number of picture saved: 3045
the number of picture saved: 3046
the number of picture saved: 3047
the number of picture saved: 3048
the number of picture saved: 3049
the number of picture saved: 3050
the number of picture saved: 3051
the number of picture saved: 3052
the number of picture saved: 3053
the number of picture saved: 3054
the number of picture saved: 3055
the number of picture saved: 3056
the number of picture saved: 3057
the number of picture saved: 3058
the number of picture saved: 3059
the number of picture saved: 3060
the number of picture saved: 3061
the number of picture saved: 3062
the number of picture saved: 3063
the number of picture saved: 3064
the number of picture saved: 3065
the number of picture saved: 3066
the number of picture saved: 3067
the number of picture saved: 3068
the number of picture saved: 3069
the number of picture saved: 3070
the number of 

the number of picture saved: 3449
the number of picture saved: 3450
the number of picture saved: 3451
the number of picture saved: 3452
the number of picture saved: 3453
the number of picture saved: 3454
the number of picture saved: 3455
the number of picture saved: 3456
the number of picture saved: 3457
the number of picture saved: 3458
the number of picture saved: 3459
the number of picture saved: 3460
the number of picture saved: 3461
the number of picture saved: 3462
the number of picture saved: 3463
the number of picture saved: 3464
the number of picture saved: 3465
the number of picture saved: 3466
the number of picture saved: 3467
the number of picture saved: 3468
the number of picture saved: 3469
the number of picture saved: 3470
the number of picture saved: 3471
the number of picture saved: 3472
the number of picture saved: 3473
the number of picture saved: 3474
the number of picture saved: 3475
the number of picture saved: 3476
the number of picture saved: 3477
the number of 

   # 生成tfrecord  格式
   generate_tfRecord
   write_tfRecord
   
   #config.data_file='/home/qw/Documents/datasets/fer2013'

In [106]:
#三个参数：目标tfrecord文件名，图像路径，图像文件名和label的列表文件。加一个调试开关
import random
from PIL import Image
def write_tfRecord(tfRecordName, image_path, label_path,is_dbg = False,is_print = False):
    writer = tf.python_io.TFRecordWriter(tfRecordName)
    num_pic = 0
    f = open(label_path, 'r')#是txt文件，存的是具体的图片文件名列表和标签，其实不存标签，从文件名也能拆出来标签。
    contents = f.readlines()#按行读出所有
    print('contents type and len is:',type(contents),len(contents))#print(contents[0])
    f.close()
    
    if is_dbg:#调试，只写一小部分数据，用来验证读写操作没有问题,
        contents = contents[:10]
    else:#如果需要看对应数据，别shuffle
        random.shuffle(contents)#原地打乱contents
        
    if is_print:#for循环内的代码，拎出来单个数据跑一下：
        content = contents[0]
        value = content.split()
        print('value is ',value)#value is  ['21276_4.png', '4']
        img_path = image_path + value[0]
        print('img_path:',img_path)#img_path: /home/qw/Documents/datasets/fer2013/train/21276_4.png
        img = Image.open(img_path).convert('L')#灰度图
        print('arr is :\n',np.array(img))
        print('img:',img)#<PIL.Image.Image image mode=L size=48x48 at 0x7F2085391710>
        img_raw = img.tobytes()#注意这个，这个直接就从img对象变成像素个数了，不存在二维需要再变换的问题。
        print(type(img_raw),'len(img_raw):',len(img_raw))#2304，像素个数，len([img_raw]==1,len([img_raw][0]==2304
        
        labels = [0] * 7#从代码看，这不是fer2013plus，plus的标签已经增加了！！！！！！！！！！！！！！！！！！！！！！！！！！
        labels[int(value[1])] = 1#one-hot的那一个1

        print('img_raw:',img_raw)#b"\x90\x81\x9e\xb0\xb1\..................."
        labels_to_int64_list = tf.train.Int64List(value=labels)
        print('Int64List is ',type(labels_to_int64_list),'\n',labels_to_int64_list)
        print('BytesList is ',tf.train.BytesList(value=[img_raw]))

        example = tf.train.Example(features = tf.train.Features(feature = {#protobuf，example打包。
            'img_raw': tf.train.Feature(bytes_list = tf.train.BytesList(value=[img_raw])),#value
            'label': tf.train.Feature(int64_list = tf.train.Int64List(value=labels))#因为是onehot，所以是个list
        }))
        #writer.write(example.SerializeToString())#不存储
        print('example is ',example)
        #反向操作，验证一下
        dict_parsed = tf.parse_single_example(example.SerializeToString(),features = {
            'img_raw':tf.FixedLenFeature([], tf.string),
            'label':tf.FixedLenFeature([7], tf.int64)
        })
        print(dict_parsed)
        label_parsed = dict_parsed['label']
        img_raw_parsed = dict_parsed['img_raw']
        with tf.Session() as sess:
            print('label parsed:',sess.run(label_parsed))
            print('img_raw_parsed parsed:',sess.run(img_raw_parsed))
    
    
        writer.close()
        return
    

    for content in contents:
        value = content.split()
        img_path = image_path + value[0]
        img = Image.open(img_path).convert('L')#灰度图
        img_raw = img.tobytes()
        labels = [0] * 7#从代码看，这不是fer2013plus，plus的标签已经增加了！！！！！！！！！！！！！！！！！！！！！！！！！！
        labels[int(value[1])] = 1#one-hot的那一个1
        example = tf.train.Example(features = tf.train.Features(feature = {
            'img_raw': tf.train.Feature(bytes_list = tf.train.BytesList(value=[img_raw])),
            'label': tf.train.Feature(int64_list = tf.train.Int64List(value=labels))
        }))
        writer.write(example.SerializeToString())
        num_pic += 1
        print('the number of picture:',num_pic)
    writer.close()

#打印调试的话，用is_dbg=True
write_tfRecord(config.tfRecord_train, config.image_train_path, config.label_train_path,is_dbg=True,is_print = True)

contents type and len is: <class 'list'> 10
value is  ['0_0.png', '0']
img_path: /home/qw/Documents/datasets/fer2013/train/0_0.png
arr is :
 [[ 70  80  82 ...,  52  43  41]
 [ 65  61  58 ...,  56  52  44]
 [ 50  43  54 ...,  49  56  47]
 ..., 
 [ 91  65  42 ...,  72  56  43]
 [ 77  82  79 ..., 105  70  46]
 [ 77  72  84 ..., 106 109  82]]
img: <PIL.Image.Image image mode=L size=48x48 at 0x7FC6B4E924A8>
<class 'bytes'> len(img_raw): 2304
img_raw: b'FPRH::<?6:<0Ysywsnb[TTZcn~\x8f\x99\x9e\xab\xa9\xac\xa9\xa5\x81nqk_OB>89=4+)A=:98EKFA86i\x92\x9a\x97\x97\x9b\x9b\x96\x93\x93\x94\x98\x9e\xa4\xac\xb1\xb6\xba\xbd\xbc\xbe\xbc\xb4\xa7t_gaMH>7:684,2+6@?GD@4Bw\x9c\xa1\xa4\xa3\xa4\xa7\xa8\xaa\xae\xaf\xb0\xb2\xb3\xb7\xbb\xbe\xc3\xc5\xc6\xc5\xc6\xc3\xbf\xbe\x91VdZA9<63)18/&,?7.467S\x8a\x9d\x9e\xa5\xa8\xac\xab\xad\xb0\xb3\xb3\xb4\xb6\xb9\xbb\xbd\xbd\xc0\xc5\xc8\xc7\xc4\xc6\xc8\xc6\xc5\xb1[W`::;3*%)/-%#$\x1e)/;^\x8d\x9f\xa1\xa1\xa4\xaa\xab\xac\xb0\xb2\xb3\xb6\xb7\xb7\xbb\xbd\xc0\xc0\xc2\xc3\xc8\xc8\xc7\

In [107]:
#def generate_tfRecord():
is_exists = os.path.exists(config.data_file)
if not is_exists:
    res = os.makedirs(config.data_file)
    print('mkdir result:',res)
else:
    print('dir already exists')
write_tfRecord(config.tfRecord_train, config.image_train_path, config.label_train_path)
write_tfRecord(config.tfRecord_valid, config.image_valid_path, config.label_valid_path)
write_tfRecord(config.tfRecord_test, config.image_test_path, config.label_test_path)

dir already exists
contents type and len is: <class 'list'> 28709
the number of picture: 1
the number of picture: 2
the number of picture: 3
the number of picture: 4
the number of picture: 5
the number of picture: 6
the number of picture: 7
the number of picture: 8
the number of picture: 9
the number of picture: 10
the number of picture: 11
the number of picture: 12
the number of picture: 13
the number of picture: 14
the number of picture: 15
the number of picture: 16
the number of picture: 17
the number of picture: 18
the number of picture: 19
the number of picture: 20
the number of picture: 21
the number of picture: 22
the number of picture: 23
the number of picture: 24
the number of picture: 25
the number of picture: 26
the number of picture: 27
the number of picture: 28
the number of picture: 29
the number of picture: 30
the number of picture: 31
the number of picture: 32
the number of picture: 33
the number of picture: 34
the number of picture: 35
the number of picture: 36
the num

the number of picture: 306
the number of picture: 307
the number of picture: 308
the number of picture: 309
the number of picture: 310
the number of picture: 311
the number of picture: 312
the number of picture: 313
the number of picture: 314
the number of picture: 315
the number of picture: 316
the number of picture: 317
the number of picture: 318
the number of picture: 319
the number of picture: 320
the number of picture: 321
the number of picture: 322
the number of picture: 323
the number of picture: 324
the number of picture: 325
the number of picture: 326
the number of picture: 327
the number of picture: 328
the number of picture: 329
the number of picture: 330
the number of picture: 331
the number of picture: 332
the number of picture: 333
the number of picture: 334
the number of picture: 335
the number of picture: 336
the number of picture: 337
the number of picture: 338
the number of picture: 339
the number of picture: 340
the number of picture: 341
the number of picture: 342
t

the number of picture: 698
the number of picture: 699
the number of picture: 700
the number of picture: 701
the number of picture: 702
the number of picture: 703
the number of picture: 704
the number of picture: 705
the number of picture: 706
the number of picture: 707
the number of picture: 708
the number of picture: 709
the number of picture: 710
the number of picture: 711
the number of picture: 712
the number of picture: 713
the number of picture: 714
the number of picture: 715
the number of picture: 716
the number of picture: 717
the number of picture: 718
the number of picture: 719
the number of picture: 720
the number of picture: 721
the number of picture: 722
the number of picture: 723
the number of picture: 724
the number of picture: 725
the number of picture: 726
the number of picture: 727
the number of picture: 728
the number of picture: 729
the number of picture: 730
the number of picture: 731
the number of picture: 732
the number of picture: 733
the number of picture: 734
t

the number of picture: 1145
the number of picture: 1146
the number of picture: 1147
the number of picture: 1148
the number of picture: 1149
the number of picture: 1150
the number of picture: 1151
the number of picture: 1152
the number of picture: 1153
the number of picture: 1154
the number of picture: 1155
the number of picture: 1156
the number of picture: 1157
the number of picture: 1158
the number of picture: 1159
the number of picture: 1160
the number of picture: 1161
the number of picture: 1162
the number of picture: 1163
the number of picture: 1164
the number of picture: 1165
the number of picture: 1166
the number of picture: 1167
the number of picture: 1168
the number of picture: 1169
the number of picture: 1170
the number of picture: 1171
the number of picture: 1172
the number of picture: 1173
the number of picture: 1174
the number of picture: 1175
the number of picture: 1176
the number of picture: 1177
the number of picture: 1178
the number of picture: 1179
the number of pictur

the number of picture: 1618
the number of picture: 1619
the number of picture: 1620
the number of picture: 1621
the number of picture: 1622
the number of picture: 1623
the number of picture: 1624
the number of picture: 1625
the number of picture: 1626
the number of picture: 1627
the number of picture: 1628
the number of picture: 1629
the number of picture: 1630
the number of picture: 1631
the number of picture: 1632
the number of picture: 1633
the number of picture: 1634
the number of picture: 1635
the number of picture: 1636
the number of picture: 1637
the number of picture: 1638
the number of picture: 1639
the number of picture: 1640
the number of picture: 1641
the number of picture: 1642
the number of picture: 1643
the number of picture: 1644
the number of picture: 1645
the number of picture: 1646
the number of picture: 1647
the number of picture: 1648
the number of picture: 1649
the number of picture: 1650
the number of picture: 1651
the number of picture: 1652
the number of pictur

...

the number of picture: 1881
the number of picture: 1882
the number of picture: 1883
the number of picture: 1884
the number of picture: 1885
the number of picture: 1886
the number of picture: 1887
the number of picture: 1888
the number of picture: 1889
the number of picture: 1890
the number of picture: 1891
the number of picture: 1892
the number of picture: 1893
the number of picture: 1894
the number of picture: 1895
the number of picture: 1896
the number of picture: 1897
the number of picture: 1898
the number of picture: 1899
the number of picture: 1900
the number of picture: 1901
the number of picture: 1902
the number of picture: 1903
the number of picture: 1904
the number of picture: 1905
the number of picture: 1906
the number of picture: 1907
the number of picture: 1908
the number of picture: 1909
the number of picture: 1910
the number of picture: 1911
the number of picture: 1912
the number of picture: 1913
the number of picture: 1914
the number of picture: 1915
the number of pictur

the number of picture: 2205
the number of picture: 2206
the number of picture: 2207
the number of picture: 2208
the number of picture: 2209
the number of picture: 2210
the number of picture: 2211
the number of picture: 2212
the number of picture: 2213
the number of picture: 2214
the number of picture: 2215
the number of picture: 2216
the number of picture: 2217
the number of picture: 2218
the number of picture: 2219
the number of picture: 2220
the number of picture: 2221
the number of picture: 2222
the number of picture: 2223
the number of picture: 2224
the number of picture: 2225
the number of picture: 2226
the number of picture: 2227
the number of picture: 2228
the number of picture: 2229
the number of picture: 2230
the number of picture: 2231
the number of picture: 2232
the number of picture: 2233
the number of picture: 2234
the number of picture: 2235
the number of picture: 2236
the number of picture: 2237
the number of picture: 2238
the number of picture: 2239
the number of pictur

the number of picture: 2538
the number of picture: 2539
the number of picture: 2540
the number of picture: 2541
the number of picture: 2542
the number of picture: 2543
the number of picture: 2544
the number of picture: 2545
the number of picture: 2546
the number of picture: 2547
the number of picture: 2548
the number of picture: 2549
the number of picture: 2550
the number of picture: 2551
the number of picture: 2552
the number of picture: 2553
the number of picture: 2554
the number of picture: 2555
the number of picture: 2556
the number of picture: 2557
the number of picture: 2558
the number of picture: 2559
the number of picture: 2560
the number of picture: 2561
the number of picture: 2562
the number of picture: 2563
the number of picture: 2564
the number of picture: 2565
the number of picture: 2566
the number of picture: 2567
the number of picture: 2568
the number of picture: 2569
the number of picture: 2570
the number of picture: 2571
the number of picture: 2572
the number of pictur

the number of picture: 2841
the number of picture: 2842
the number of picture: 2843
the number of picture: 2844
the number of picture: 2845
the number of picture: 2846
the number of picture: 2847
the number of picture: 2848
the number of picture: 2849
the number of picture: 2850
the number of picture: 2851
the number of picture: 2852
the number of picture: 2853
the number of picture: 2854
the number of picture: 2855
the number of picture: 2856
the number of picture: 2857
the number of picture: 2858
the number of picture: 2859
the number of picture: 2860
the number of picture: 2861
the number of picture: 2862
the number of picture: 2863
the number of picture: 2864
the number of picture: 2865
the number of picture: 2866
the number of picture: 2867
the number of picture: 2868
the number of picture: 2869
the number of picture: 2870
the number of picture: 2871
the number of picture: 2872
the number of picture: 2873
the number of picture: 2874
the number of picture: 2875
the number of pictur

the number of picture: 3160
the number of picture: 3161
the number of picture: 3162
the number of picture: 3163
the number of picture: 3164
the number of picture: 3165
the number of picture: 3166
the number of picture: 3167
the number of picture: 3168
the number of picture: 3169
the number of picture: 3170
the number of picture: 3171
the number of picture: 3172
the number of picture: 3173
the number of picture: 3174
the number of picture: 3175
the number of picture: 3176
the number of picture: 3177
the number of picture: 3178
the number of picture: 3179
the number of picture: 3180
the number of picture: 3181
the number of picture: 3182
the number of picture: 3183
the number of picture: 3184
the number of picture: 3185
the number of picture: 3186
the number of picture: 3187
the number of picture: 3188
the number of picture: 3189
the number of picture: 3190
the number of picture: 3191
the number of picture: 3192
the number of picture: 3193
the number of picture: 3194
the number of pictur

# 最后是提供过训练过程使用的读取tfrecord的接口
get_tfrecord和read_tfRecord接口在下方，这里拆开来为了调试。

跑神经网络跑不通，所以单独拿出来调一下。

In [6]:
#不shuffle的版本，已通
import tensorflow as tf
import fer_config as config
from PIL import Image

#生产者通道
tfRecord_path = config.tfRecord_train
filename_queue = tf.train.string_input_producer([tfRecord_path])#文件名队列
reader = tf.TFRecordReader()
tmp, serialized_example = reader.read(filename_queue)#从文件队列拿文件名，读取文件，这个时候给返回的k-v对，k到底指什么？v是dict？
print(type(serialized_example))
features = tf.parse_single_example(serialized_example,#解析
                                  features={#和存储对应
                                      'label':tf.FixedLenFeature([7],tf.int64),
                                      'img_raw':tf.FixedLenFeature([], tf.string)
                                  })
img_not_decode = features['img_raw']
img = tf.decode_raw(features['img_raw'],tf.uint8)#之前Img对象被按像素转成bytes（属于平铺）又转成二进制存储了。不过读出来首先是Tensor，接口也得用tf
print(type(img))
img.set_shape([config.img_height * config.img_height])
img_not_cast = img
img_final = tf.cast(img, tf.float32) * (1./255)#数值成了0~1，为了训练收敛快
label = tf.cast(features['label'], tf.float32)
# return img, label
print('filename_queue:',filename_queue)
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    

    coord=tf.train.Coordinator()
    threads=tf.train.start_queue_runners(sess=sess,coord=coord)
    for i in range(2):
    #     print('not decode:',sess.run(img_not_decode))
        print('label:',sess.run(label))#无下标，因为一次取的就是一个数据
        print(sess.run(img_not_cast))
        img_arr = sess.run(img)
        print(type(img_arr),img_arr.shape)
        img_arr = img_arr.reshape(48,48)
        print(type(img_arr),img_arr.shape)
        pil_img = Image.fromarray(img_arr)
        pil_img.show()

        print(sess.run(img_final))
        #这个返回值，文件加行号吧#b'/home/qw/Documents/datasets/fer2013/fer2013_train.tfrecords:18928'
        print('tmp is :',sess.run(tmp))
#     print(sess.run(filename_queue))
    coord.request_stop()
    coord.join(threads)    


<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
filename_queue: <tensorflow.python.ops.data_flow_ops.FIFOQueue object at 0x7f714002d860>
label: [ 0.  0.  0.  0.  1.  0.  0.]
[114 157 197 ...,  66  25  29]
<class 'numpy.ndarray'> (2304,)
<class 'numpy.ndarray'> (48, 48)
[ 0.92549026  0.92941183  0.83921576 ...,  0.25098041  0.44705886
  0.60000002]
tmp is : b'/home/qw/Documents/datasets/fer2013/fer2013_train.tfrecords:9464'
label: [ 0.  0.  0.  0.  0.  0.  1.]
[214 218 240 ...,  67 139 143]
<class 'numpy.ndarray'> (2304,)
<class 'numpy.ndarray'> (48, 48)
[ 0.74901962  0.71764708  0.67058825 ...,  0.24705884  0.26274511
  0.2392157 ]
tmp is : b'/home/qw/Documents/datasets/fer2013/fer2013_train.tfrecords:21294'


In [2]:
#shuffle的版本，已通
import tensorflow as tf
import fer_config as config
from PIL import Image

#生产者通道
tfRecord_path = config.tfRecord_train
filename_queue = tf.train.string_input_producer([tfRecord_path])#文件名队列
reader = tf.TFRecordReader()
tmp, serialized_example = reader.read(filename_queue)#从文件队列拿文件名，读取文件，这个时候给返回的k-v对，k到底指什么？v是dict？
print(type(serialized_example))
features = tf.parse_single_example(serialized_example,#解析
                                  features={#和存储对应
                                      'label':tf.FixedLenFeature([7],tf.int64),
                                      'img_raw':tf.FixedLenFeature([], tf.string)
                                  })
img_not_decode = features['img_raw']
img = tf.decode_raw(features['img_raw'],tf.uint8)#之前Img对象被按像素转成bytes（属于平铺）又转成二进制存储了。不过读出来首先是Tensor，接口也得用tf
print(type(img))
img.set_shape([config.img_height * config.img_height])
img_not_cast = img
img_final = tf.cast(img, tf.float32) * (1./255)#数值成了0~1，为了训练收敛快
img = img_final#还原原来的流程
label = tf.cast(features['label'], tf.float32)
# return img, label
img_batch, label_batch = tf.train.shuffle_batch(
    [img, label], batch_size = 2, num_threads = 2,capacity = 10000, min_after_dequeue=5000)#一次取两个

print('filename_queue:',filename_queue)
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    

    coord=tf.train.Coordinator()
    threads=tf.train.start_queue_runners(sess=sess,coord=coord)

    print('label:',sess.run(label))
    img_arr = sess.run(img_batch[0])
    print(type(img_arr),img_arr.shape)
    img_arr = img_arr.reshape(48,48)
    print(type(img_arr),img_arr.shape)
    pil_img = Image.fromarray(img_arr)
    pil_img.show()

    img_arr = sess.run(img_batch[1])#从batch拿，也没问题
    print(type(img_arr),img_arr.shape)
    img_arr = img_arr.reshape(48,48)
    print(type(img_arr),img_arr.shape)
    pil_img = Image.fromarray(img_arr)
    pil_img.show()
    print(sess.run(img_final))
    print('tmp is :',sess.run(tmp))
    
    coord.request_stop()
    coord.join(threads)    


<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
filename_queue: <tensorflow.python.ops.data_flow_ops.FIFOQueue object at 0x7f8a14402748>
label: [ 0.  0.  0.  0.  0.  0.  1.]
<class 'numpy.ndarray'> (2304,)
<class 'numpy.ndarray'> (48, 48)
<class 'numpy.ndarray'> (2304,)
<class 'numpy.ndarray'> (48, 48)
[ 0.45490199  0.48235297  0.49803925 ...,  0.627451    0.60784316
  0.34117648]
tmp is : b'/home/qw/Documents/datasets/fer2013/fer2013_train.tfrecords:12097358'


In [6]:
#shuffle的版本，换一种写法：img和label两个个batch合并起来一次读出
#实测：裸写都没问题
import tensorflow as tf
import fer_config as config
from PIL import Image

#生产者通道
tfRecord_path = config.tfRecord_train
filename_queue = tf.train.string_input_producer([tfRecord_path])#文件名队列
reader = tf.TFRecordReader()
tmp, serialized_example = reader.read(filename_queue)#从文件队列拿文件名，读取文件，这个时候给返回的k-v对，k到底指什么？v是dict？
print(type(serialized_example))
features = tf.parse_single_example(serialized_example,#解析
                                  features={#和存储对应
                                      'label':tf.FixedLenFeature([7],tf.int64),
                                      'img_raw':tf.FixedLenFeature([], tf.string)
                                  })
img_not_decode = features['img_raw']
img = tf.decode_raw(features['img_raw'],tf.uint8)#之前Img对象被按像素转成bytes（属于平铺）又转成二进制存储了。不过读出来首先是Tensor，接口也得用tf
print(type(img))
img.set_shape([config.img_height * config.img_height])
img_not_cast = img
img_final = tf.cast(img, tf.float32) * (1./255)#数值成了0~1，为了训练收敛快
# img = img_final#还原原来的流程，要打印图像就屏蔽这句
label = tf.cast(features['label'], tf.float32)
# return img, label
img_batch, label_batch = tf.train.shuffle_batch(
    [img, label], batch_size = 2, num_threads = 2,capacity = 10000, min_after_dequeue=5000)#一次取两个

print('filename_queue:',filename_queue)
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    

    coord=tf.train.Coordinator()
    threads=tf.train.start_queue_runners(sess=sess,coord=coord)

    print('label:',sess.run(label))
    #分解下一个例子的错误代码
    xs= sess.run(img_batch)
    ys = sess.run(label_batch)
    print(type(xs),xs.shape)#2*2304，也对啊
    print(xs)
    print(xs[0])#这个操作本身也对啊
    print('label:',ys[0])#这个操作本身也对啊
    xs0 = xs[0].reshape(48,48)
    xs0_img = Image.fromarray(xs0)
    xs0_img.show()#打印也没问题
    
    #错在这个中括号加两个tensor？不该
    xs, ys = sess.run([img_batch, label_batch])
    print(type(xs),xs.shape)
    print(xs[0])#这个操作本身也对啊
    print('label:',ys[0])#这个操作本身也对啊
    
    coord.request_stop()
    coord.join(threads)    


<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
filename_queue: <tensorflow.python.ops.data_flow_ops.FIFOQueue object at 0x7f8a14328208>
label: [ 0.  0.  0.  0.  0.  1.  0.]
<class 'numpy.ndarray'> (2, 2304)
[[124 136 126 ..., 177 172 166]
 [123 133 139 ...,  46  49  51]]
[124 136 126 ..., 177 172 166]
label: [ 0.  0.  0.  0.  0.  1.  0.]
<class 'numpy.ndarray'> (2, 2304)
[255 254 218 ..., 222 254 253]
label: [ 1.  0.  0.  0.  0.  0.  0.]


In [ ]:
##前边实测：一步步拆解，裸写都没问题，作为两个函数互相调用就出问题
#最终确认，问题出在生产者的定义，放在了线程协调器启动之后，裸写的代码等于先调用了。

import tensorflow as tf
import fer_config as config
from PIL import Image
def read_tfRecord(tfRecord_path):
    filename_queue = tf.train.string_input_producer([tfRecord_path])#文件名队列
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)#从文件队列拿文件名，读取文件
    features = tf.parse_single_example(serialized_example,#解析
                                      features={#和存储对应
                                          'label':tf.FixedLenFeature([7],tf.int64),
                                          'img_raw':tf.FixedLenFeature([], tf.string)
                                      })
    img = tf.decode_raw(features['img_raw'],tf.uint8)#之前Img对象被按像素转成bytes（属于平铺）又转成二进制存储了。不过读出来首先是Tensor，接口也得用tf
    print(type(img))
    img.set_shape([config.img_height * config.img_height])
    
    img = tf.cast(img, tf.float32) * (1./255)#浮点数，图像是整型的，浮点为了训练精度高吧。
    label = tf.cast(features['label'], tf.float32)
    return img, label

#指定路径和数量，随机产生batch
def get_tfrecord(num, tfRecord_path):
    img, label = read_tfRecord(tfRecord_path)
    img_batch, label_batch = tf.train.shuffle_batch([img, label], batch_size = num, num_threads = 2,
                                                   capacity = 10000, min_after_dequeue=5000)
    return img_batch, label_batch  
#     return img,label
img_batch,label_batch=get_tfrecord(2,config.tfRecord_train)#从tfrecord拿数据。
with tf.Session() as sess:
    tf.global_variables_initializer().run()

    
    coord=tf.train.Coordinator()
    threads=tf.train.start_queue_runners(sess=sess,coord=coord)
    
    #img_batch,label_batch=get_tfrecord(2,config.tfRecord_train)#这个位置是血的教训
    print('img_batch:',img_batch)
    xs, ys = sess.run([img_batch,label_batch])
#     print(xs[0])
#     print(ys[0])

    coord.request_stop()
    coord.join(threads)    

<class 'tensorflow.python.framework.ops.Tensor'>
img_batch: Tensor("shuffle_batch_2:0", shape=(2, 2304), dtype=float32)


In [ ]:
# help(tf.TFRecordReader().read)#这个拿到的是key，value？反正是datasets！
#但是目前的两个版本，一个是要自己写一个映射，一个是用producer，都没有裸操作。
#回头再实现裸reader的

# help(pd.core.frame.DataFrame) pandas数据结构
#二维大小可变，可能是异构的表格数据结构，带有标记的轴（行和列）。算术运算同时在行和列标签上对齐align。
#可以将其视为序列对象的类似dict的容器。pandas原始数据结构

In [47]:
ts1 = 10
ts2 = 20
ts3 = 19
ts4 = 33
d = {'col1': ts1, 'col2': ts2, 'col3':ts3, 'col4':ts4}

#排序依据index，index是这里生成的，前边是列，其实列才是数据，一列对应一个key，行应该是属性
#好像本例读取csv，列确实是属性，行才是数据。
index = np.arange(4)
print(index)
df = pd.core.frame.DataFrame(data=d, index=index)#默认是np.arange(n)
print('df is ',df)

#列不变，影响行的顺序，感觉可以用来随机排列数据
index2 = [1,0,2]
print(index2)
df1 = pd.core.frame.DataFrame(data=d, index=index2)#默认是np.arange(n)
print('df1 is ',df1)

#实际操作：排序数据,排列的依据是下标
#第一个例子：没用，因为本来就是顺序的
index3 = index[np.argsort(index)]
print(index3)
df11 = pd.core.frame.DataFrame(data=d, index=index3)#默认是np.arange(n)
print('df11 is ',df11)

#第二个例子，用index2来sort，因为那个顺序相对不是那么“顺”
index4 = index[np.argsort(index2)]
print(index4)
df12 = pd.core.frame.DataFrame(data=d, index=index3)#默认是np.arange(n)
print('df12 is ',df12)

#第三个例子，随机排列
print(np.random.shuffle(index))
print(index)
df13 = pd.core.frame.DataFrame(data=d, index=index)#默认是np.arange(n)
print('df13 is ',df13)

#小结：不过这个DataFrame本来就是生成数据的，这里都是新生成的一样的数据。
#单纯看此处，打乱下标不是很有用，除非打乱下标再填充数据再排序，不如有数据再shuffle。


[0 1 2 3]
df is     col1  col2  col3  col4
0    10    20    19    33
1    10    20    19    33
2    10    20    19    33
3    10    20    19    33
[1, 0, 2]
df1 is     col1  col2  col3  col4
1    10    20    19    33
0    10    20    19    33
2    10    20    19    33
[0 1 2 3]
df11 is     col1  col2  col3  col4
0    10    20    19    33
1    10    20    19    33
2    10    20    19    33
3    10    20    19    33
[1 0 2]
df12 is     col1  col2  col3  col4
0    10    20    19    33
1    10    20    19    33
2    10    20    19    33
3    10    20    19    33
None
[3 1 0 2]
df13 is     col1  col2  col3  col4
3    10    20    19    33
1    10    20    19    33
0    10    20    19    33
2    10    20    19    33
df2:           0         1         2         3         4
0  0.124829  1.418002 -0.519830  1.282067  1.654994
1 -1.277467 -0.228165 -0.667777  1.821163  0.373094
2 -0.519576 -0.194437  1.954480 -0.998827  1.484214
3 -0.883194  0.738878 -1.309969 -0.752936  0.400168
4 -0.850941  0.5

In [101]:
#单纯随机生成，5列4行，列才是数据，行是属性特征
df2 = pd.core.frame.DataFrame(np.random.randn(4, 3))
print('df2:',df2)
#改一下，自定义列名
df3 = pd.core.frame.DataFrame(np.random.randn(4, 3),columns=['a', 'b', 'c'])
print('df3:',df3)

#索引是直观打印的列，但是是二维索引的“行”，是key。

#单个网格操作
df3['b'][0] = 10
df3['c'][2] = 1000
print('after assign :\n',df3)
#按列广播操作
df3['a'] += 1000
print('after broadcast += 1000:\n',df3)
#字典形式，没法按行广播
# df3[:][2] += 1000
# print('broadcast +=10000:\n',df3)

df2:           0         1         2
0 -0.624940  0.786240 -0.332486
1 -1.311156  0.111817 -0.295657
2 -0.247502 -0.235453  0.473848
3 -0.703555 -0.623886 -2.954247
df3:           a         b         c
0 -1.490993 -2.515879 -1.352138
1 -0.342885  0.485260 -0.332244
2  0.631913  0.656932 -0.005665
3 -0.479486  1.550015  0.068343
after assign :
           a          b            c
0 -1.490993  10.000000    -1.352138
1 -0.342885   0.485260    -0.332244
2  0.631913   0.656932  1000.000000
3 -0.479486   1.550015     0.068343
after broadcast += 1000:
              a          b            c
0   998.509007  10.000000    -1.352138
1   999.657115   0.485260    -0.332244
2  1000.631913   0.656932  1000.000000
3   999.520514   1.550015     0.068343


In [62]:
#回到主题，我读取了三万多数据，没有key（也就是文件地址），我就没法只列出部分数据（行）了？
#好像在csv，列才是特征？行是数据？其实不是，这个原始数据的csv还谈不上特征！特征作为一个object，全在列pixels内。

#emotion是label， pixels是图像， Usage是阶段的选择
# Usage=Training应该是字符串还是转了boolean？读出来都是字符串，后续需要自己转。
print(data['emotion'][0:5])#
print(data['Usage'][0:2])#

#csv刚读出来都是字符串，还不是正经的像素
# print(data['pixels'][0].shape)#AttributeError: 'str' object has no attribute 'shape'
#每张图还有区别，不是规整的，肯定需要处理
#他们都不等长，因为字符串“13 44 65”和"133 222 111"不一样长，这是纯字符串。
print(len(data['pixels'][0]))
print(len(data['pixels'][1]))
print(len(data['pixels'][2]))

0    0
1    0
2    2
3    4
4    6
Name: emotion, dtype: int64
0    Training
1    Training
Name: Usage, dtype: object
8287
8687
8581


# 复制fromarray代码，处理——关于  <i8  的报错
注意，很多模块有重名的，必须用Image.

In [4]:
def fromarray(obj, mode=None):
    """
    从导出array interface的obj创建图像内存（使用protobuf协议）。
    如果obj不是连续的，则调用tobytes方法并使用：py:func:`~pil.image.frombuffer`。 
    :param obj: 带obj.__array_interface__的
    :param mode: Mode to use (will be determined from type if None)
      See: :ref:`concept-modes`.
    :returns: An image object.
    """
    arr = obj.__array_interface__#<class 'dict'>  其实是一个字典打包输入！！！obj自己实现的接口
    shape = arr['shape']#字典中的key，得到图像shape
    ndim = len(shape)#维度，（48*48的灰度图是二维，有颜色多通道就是三维。）
    print('in fromarray :Image._fromarray_typemap is :',Image._fromarray_typemap)
    strides = arr.get('strides', None)#没有strides就给赋值None
    if mode is None:
        try:
            typekey = (1, 1) + shape[2:], arr['typestr']
            mode, rawmode = Image._fromarray_typemap[typekey]#这是从一个typemap找匹配出错了
        except KeyError:
            print(typekey)
            raise TypeError("Cannot handle this data type")
    else:
        rawmode = mode
    if mode in ["1", "L", "I", "P", "F"]:
        ndmax = 2
    elif mode == "RGB":
        ndmax = 3
    else:
        ndmax = 4
    if ndim > ndmax:
        raise ValueError("Too many dimensions: %d > %d." % (ndim, ndmax))

    size = shape[1], shape[0]
    if strides is not None:
        if hasattr(obj, 'tobytes'):
            obj = obj.tobytes()
        else:
            obj = obj.tostring()

    return Image.frombuffer(mode, size, obj, "raw", rawmode, 0, 1)


## 解决方法，np.array的dtype参数问题，可能之前默认int，我是64位系统，就是int64，指定int32即可。

In [ ]:
#二次转存之前，从原始csv读出来，也是<i8
print(type(data_train))
print((data_train.shape))
print((data_train['emotion'][0]))
print((type(data_train['pixels'][0])))

labels = data['emotion'][0]
p = data_train['pixels'][0]
# print(p)

images = np.array(p.split(" "),dtype= int)
print(images.shape)
images = np.reshape(images, (config.img_width, config.img_height))
print(images.shape)

print(images.__array_interface__)

images2 = np.array(p.split(" "),dtype = np.int32)
images3 = np.array(p.split(" "),dtype = np.int64)
images4 = np.array(p.split(" "),dtype = '<i4')
print(images2.__array_interface__)
print(images3.__array_interface__)
print(images4.__array_interface__)



# test_data_mod = fromarray(images)

In [37]:
#关于np.array里边带列参数的例子
x = np.array([(1,2),(3,4)],dtype=[('a','<i4'),('b','<i4')])
print(x['a'])
print(x)
#列？也不算列吧，输入是tuple,打印也是tuple，但是算是生成两个array，可能某些场合某种数据需要
#至少后边那个a和b算是增加了一种按“列”访问的方式，本质上还是array里的tuple
x = np.array([(1,2),(3,4), (5,6)],dtype=[('a','<i4'),('b','<i8')])
print('x[0]:',x[0])#传统二维数组的访问也可
print('x[0][0]:',x[0][0])
print(x['a'][0])#列的访问也可
print(x.shape)
print('type x:',type(x))
print(type(x['a']))
print(type(x['a'][0]))
print(x['b'][0])
print(type(x['b']))
print(type(x['b'][0]))
print(x)
#其实就是数组中的tuple
x = np.array([(1,2),(3,4)])
print('access tuple:',x[0])
print('type x[0]:',type(x[0]))
print(x)
# x = np.array([[1,2],[3,4]],dtype=[('a','<i4'),('b','<i4')])#TypeError: a bytes-like object is required, not 'int'
# x = np.array([1,2],dtype=[('a','<i4'),('b','<i4')])#TypeError: a bytes-like object is required, not 'int'
# print(x['a'])
# print(x)

help(np.array)

[1 3]
[(1, 2) (3, 4)]
x[0]: (1, 2)
x[0][0]: 1
1
(3,)
type x: <class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.int32'>
2
<class 'numpy.ndarray'>
<class 'numpy.int64'>
[(1, 2) (3, 4) (5, 6)]
access tuple: [1 2]
type x[0]: <class 'numpy.ndarray'>
[[1 2]
 [3 4]]
Help on built-in function array in module numpy.core.multiarray:

array(...)
    array(object, dtype=None, copy=True, order='K', subok=False, ndmin=0)
    
    Create an array.
    
    Parameters
    ----------
    object : array_like
        An array, any object exposing the array interface, an object whose
        __array__ method returns an array, or any (nested) sequence.
    dtype : data-type, optional
        The desired data-type for the array.  If not given, then the type will
        be determined as the minimum type required to hold the objects in the
        sequence.  This argument can only be used to 'upcast' the array.  For
        downcasting, use the .astype(t) method.
    copy : bool, optional
       

# 关于这个Interface和typestr
https://docs.scipy.org/doc/numpy/reference/arrays.interface.html
The Array Interface

什么Cython Numpy教程，支持早于python 2.6 的protobuf

version:	3

array interface，也叫array protocol。
给类似array的python对象的一种互相重用data buffers的手段

同构N维array interface 是对象共享N维数组内存和信息的默认机制。（两语言互通）
接口由一个Python端和一个使用两个属性的C端组成。想支持这个机制的对象都得提供相应属性。
Objects wishing to support an N-dimensional array in application code should look for at least one of these attributes and use the information provided appropriately.

这个interface描述了同构数组，即数组的每个项都具有相同的“type”。这种类型可以非常简单，也可以是非常任意和复杂的C型结构。
使用interface有两种方法，python和C都有，C好像是结构体，python是dict吧。

## python端
包含属性__array_interface__的对象，前边已经打印看了，不陌生。

__array_interface__：
dict，3个必须参数，5个可选参数。没传的参数有默认实现，这个可以从代码看到。
### 介绍具体的key
#### shape (required)
没什么说的，数组的形状，整型的，有可能有大数问题，需要抛出异常或者换个类型。
#### typestr (required)
三部分组成的描述：
内存形式吧：大端>   小端<   不相关|
类型：比如i是整型，f浮点
字节数：比如i4和i8
（疑问：我那是读出来的字符串吧？这是描述字符串的？整型字符串和浮点字符串？还是从字符串转成数据以后的整型？这是描述array的，本例的array是我从str中split再reshape得到的。）
#### descr
描述内存分布的，不是必须的，有些typestr用到
唯一的要求是typestr键中表示的字节数与此处表示的字节总数相同。（这个好像见到了默认是这种：[（''，typestr）]）
#### data
二元组，第一个可能是存储区域的地址或者id，第二个，True只读，两个例子都是False。
#### strides
可选，估计就是切数组的，目前多是用完整的，不管。




# 各个接口

help(random.shuffle)
a = [1,2,3,4,5]
b = random.shuffle(a)
print(b)
print(a)
help(tf.train.Example)
help(tf.train.Features)
help(int)

In [69]:
help(Image.Image.tobytes)#img的tobytes

Help on function tobytes in module PIL.Image:

tobytes(self, encoder_name='raw', *args)
    Return image as a bytes object.
    
    .. warning::
    
        This method returns the raw image data from the internal
        storage.  For compressed image data (e.g. PNG, JPEG) use
        :meth:`~.save`, with a BytesIO parameter for in-memory
        data.
    
    :param encoder_name: What encoder to use.  The default is to
                         use the standard "raw" encoder.
    :param args: Extra arguments to the encoder.
    :rtype: A bytes object.



In [ ]:
help(np.array)

In [10]:
n1 = 123
print(type(n1))
n2 = 2**63 -1#一样不是long，反正我的int已经是i8了
print(type(n2))
print(n2)
print((int))
# print(n2.__array_interface__)

#help(int)#没说32位和64位的事

<class 'int'>
<class 'int'>
9223372036854775807
<class 'int'>


In [11]:
import PIL
from PIL import Image
help(PIL)
help(Image)

Help on package PIL:

NAME
    PIL

DESCRIPTION
    # The Python Imaging Library.
    # $Id$
    #
    # package placeholder
    #
    # Copyright (c) 1999 by Secret Labs AB.
    #
    # See the README file for information on usage and redistribution.
    #

PACKAGE CONTENTS
    BdfFontFile
    BmpImagePlugin
    BufrStubImagePlugin
    ContainerIO
    CurImagePlugin
    DcxImagePlugin
    DdsImagePlugin
    EpsImagePlugin
    ExifTags
    FitsStubImagePlugin
    FliImagePlugin
    FontFile
    FpxImagePlugin
    FtexImagePlugin
    GbrImagePlugin
    GdImageFile
    GifImagePlugin
    GimpGradientFile
    GimpPaletteFile
    GribStubImagePlugin
    Hdf5StubImagePlugin
    IcnsImagePlugin
    IcoImagePlugin
    ImImagePlugin
    Image
    ImageChops
    ImageCms
    ImageColor
    ImageDraw
    ImageDraw2
    ImageEnhance
    ImageFile
    ImageFilter
    ImageFont
    ImageGrab
    ImageMath
    ImageMode
    ImageMorph
    ImageOps
    ImagePalette
    ImagePath
    ImageQt
    Image

# fromarray单独例子
PIL_fromarray_test.ipynb中，用Image.open()得到，类型  |u1